In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [3]:
samp_sub = pd.read_csv('sample_submission.csv')

In [8]:
train =  pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
y_train = train.iloc[:,-1]
train.loc[:, 'sample'] = 'train'
test.loc[:, 'sample'] = 'test'
data = train.drop('target',axis = 1).append(test).reset_index(drop=True)

In [9]:
df = data

In [10]:
df.loc[df['pdays']==999, 'pdays'] = 90
df.loc[df['duration']>1000, 'duration'] = 1000

In [11]:
def obj_lst_contructor(data):
    obj_lst = []
    for i in data.columns: # перебираем все столбцы
        if str(data[i].dtype) == 'object' and i != '_id' and i != 'sample':
            obj_lst.append(i)
    return obj_lst

obj_lst = obj_lst_contructor(df)

In [12]:
def column_deleter(df_input,i):
    df_output = df_input.copy()
    df_output = df_output.drop(i,axis = 1)
    obj_lst = obj_lst_contructor(df_output)
    print(obj_lst)
    df_output = pd.get_dummies(df_output, columns = obj_lst)
    return df_output
df_cd = column_deleter(df, df.columns[1])
df_cd.info()

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 64 columns):
_id                              41188 non-null object
duration                         41188 non-null int64
campaign                         41188 non-null int64
pdays                            41188 non-null int64
previous                         41188 non-null int64
emp.var.rate                     41188 non-null float64
cons.price.idx                   41188 non-null float64
cons.conf.idx                    41188 non-null float64
euribor3m                        41188 non-null float64
nr.employed                      41188 non-null float64
sample                           41188 non-null object
job_admin.                       41188 non-null uint8
job_blue-collar                  41188 non-null uint8
job_entrepreneur                 41188 non-null uint8
job_housemaid 

In [13]:
def get_knn_roc_auc(df_train_preproc, y_train):
    knn = KNeighborsClassifier(n_neighbors=290, weights='distance', p=1)
    cross_val_arr = cross_val_score(
        knn,
        df_train_preproc,
        y_train,
        scoring='roc_auc',
        cv=3
    )
    mean_roc_auc = np.mean(cross_val_arr)
    return mean_roc_auc #, mean_roc_auc

In [14]:
##### запускать это

drop_col_list=[]
for i in df.columns[1:-1]:
    df_cd = column_deleter(df, i)
    df_train_preproc = df_cd[df_cd['sample'] == 'train'].drop(['_id', 'sample'],axis = 1)
    cvs = get_knn_roc_auc(df_train_preproc, y_train)
#     cvs, mean_cvs = get_knn_roc_auc(df_train_preproc, y_train)
    some_list = [i, cvs.tolist()]
    drop_col_list.append(some_list)
#     drop_col_list[i].append(mean_cvs)


['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
drop_col_list